In [6]:
# Get html from url

import requests

url = 'https://koho.ca'  # Replace with the company's URL
response = requests.get(url)
html_content = response.text

with open('temp.html', 'w') as file:
    file.write(html_content)

In [7]:
# Extract text from html

from bs4 import BeautifulSoup

def extract_text_from_html(html_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract all text from the HTML
    all_text = soup.get_text(separator='\n')

    return all_text

html_file = 'temp.html'  # Replace with your HTML file path
extracted_text = extract_text_from_html(html_file)
with open('temp.txt', 'w') as file:
  file.write(extracted_text)


In [8]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model for summarization
model_name = "facebook/bart-large-cnn"  # Example model with large sequence length
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Function to summarize text
def summarize_text(text, max_input_length=1024, max_output_length=150, min_output_length=30):
    # Tokenize and truncate input text
    inputs = tokenizer(text, max_length=max_input_length, truncation=True, return_tensors="pt")

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=max_output_length, min_length=min_output_length, num_beams=2, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Generate summary
generated_summary = summarize_text(extracted_text)

# Print generated summary
print("Generated Summary:")
print(generated_summary)


Generated Summary:
KOHO is a no-fee spending and savings account where you can get instant cash back and earn interest on your entire account. You get a prepaid reloadable Mastercard that gives you all the spending power of a credit card without any fees or interest. It’s connected to a smart budgeting app that simplifies day-to-day finances.
